# 策略结果分析

## 合成回测策略结果（2023-12-14——2025-2-14）

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

result_metric = []
pairs = ["IFIH", "ICIF", "IMIC", "IMIH"]
for pair in pairs:
    df_metric = pd.read_csv("{}_concat_metrics.csv".format(pair))
    result_metric.append({
        "组合": pair,
        "累计收益率": str(round(df_metric.loc[df_metric["name"] == "total_return_pct", "value"].values[0], 2)) + "%",
        "年化收益率": str(round(df_metric.loc[df_metric["name"] == "annual_return_pct", "value"].values[0], 2)) + "%",
        "年化波动": str(round(df_metric.loc[df_metric["name"] == "annual_volatility_pct", "value"].values[0], 2)) + "%",
        "夏普比": str(round(df_metric.loc[df_metric["name"] == "sharpe", "value"].values[0], 2)),
        "卡玛比": str(round(df_metric.loc[df_metric["name"] == "calmar", "value"].values[0], 2)),
        "最大回撤": str(round(df_metric.loc[df_metric["name"] == "max_drawdown_pct", "value"].values[0], 2)) + "%",
    }
    )

df = pd.DataFrame(result_metric)
from tabulate import tabulate

print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False))


+------+------------+------------+----------+--------+--------+----------+
| 组合 | 累计收益率 | 年化收益率 | 年化波动 | 夏普比 | 卡玛比 | 最大回撤 |
+------+------------+------------+----------+--------+--------+----------+
| IFIH |   4.08%    |   3.66%    |   4.7%   |  0.77  |  1.38  |  -2.58%  |
| ICIF |   18.68%   |   16.66%   |  12.35%  |  1.33  |  2.62  |  -5.95%  |
| IMIC |   13.85%   |   12.38%   |  8.23%   |  1.42  |  1.73  |  -6.53%  |
| IMIH |   33.22%   |   29.46%   |  18.5%   |  1.33  |  1.63  | -13.95%  |
+------+------------+------------+----------+--------+--------+----------+


In [11]:
port_files = ['IFIH_concat_portfolio.csv', 'ICIF_concat_portfolio.csv', 'IMIC_concat_portfolio.csv','IMIH_concat_portfolio.csv']
result = []
for port_file in port_files:
    df = pd.read_csv(port_file)
    df['portfolio_daily_return_{}'.format(port_file[:4])] = df['market_value'].pct_change()
    df.set_index('date', inplace=True)
    result.append(df[["portfolio_daily_return_{}".format(port_file[:4])]])
df_concat = pd.concat(result, axis=1)
df_concat["portfolio_daily_return"] = df_concat["portfolio_daily_return_IFIH"] * 0.25 + df_concat["portfolio_daily_return_ICIF"] * 0.25 + \
    df_concat["portfolio_daily_return_IMIC"] * 0.25 + df_concat["portfolio_daily_return_IMIH"] * 0.25
# df_concat["date"] = pd.to_datetime(df_concat.index)
df_concat.fillna(0, inplace=True)
df_concat.index = pd.to_datetime(df_concat.index)
df_concat

,portfolio_daily_return_IFIH,portfolio_daily_return_ICIF,portfolio_daily_return_IMIC,portfolio_daily_return_IMIH,portfolio_daily_return
date,,,,,
2023-12-14,0.000000,0.000000,0.000000,0.000000,0.000000
2023-12-15,0.004656,-0.000911,-0.006199,0.000000,-0.000614
2023-12-18,-0.002727,0.005916,0.003270,0.000000,0.001615
2023-12-19,-0.003809,-0.000207,-0.003513,-0.000358,-0.001972
2023-12-20,0.001044,0.000665,0.005507,0.004448,0.002916
...,...,...,...,...,...
2025-02-10,-0.003525,0.006301,0.001034,0.003811,0.001905
2025-02-11,-0.001034,0.000271,0.001222,0.000459,0.000230
2025-02-12,-0.001213,0.004262,0.000143,0.003191,0.001596


In [8]:
df_concat.index

DatetimeIndex(['2023-12-14', '2023-12-15', '2023-12-18', '2023-12-19',
               '2023-12-20', '2023-12-21', '2023-12-22', '2023-12-25',
               '2023-12-26', '2023-12-27',
               ...
               '2025-01-24', '2025-01-27', '2025-02-05', '2025-02-06',
               '2025-02-07', '2025-02-10', '2025-02-11', '2025-02-12',
               '2025-02-13', '2025-02-14'],
              dtype='datetime64[ns]', name='date', length=280, freq=None)

In [5]:
def factor_evaluation(factors):
    """
    对多个因子进行评估，计算各项绩效指标
    
    参数:
    factors: DataFrame，包含多个因子的日收益率数据
    
    返回:
    indicators_df: DataFrame，包含所有因子的绩效指标
    """
    # 定义评估指标列表，这些将作为结果DataFrame的索引
    indexs = [
        '绝对收益率',
        '年化收益率',
        '年化波动率',
        '最大回撤',
        'Sharpe比率',
        'Calmar比率',
        'Sortino比率',
        '日胜率',
        '日盈亏比',
        '周胜率',
        '周盈亏比',
        '月胜率',
        '月盈亏比'
        # '换手率'
    ]

    # 创建空的DataFrame用于存储结果，行索引为评估指标
    indicators_df = pd.DataFrame(index=indexs)

    # 获取所有因子的名称列表
    factors_list = list(factors.columns)
    
    # 对每个因子进行评估
    for factor in factors_list:
        # 获取当前因子的日收益率数据
        factor_daily_return = factors[[factor]]

        # 计算绝对收益率（总收益率）：将所有日收益率加1后连乘，再减1
        absolute_return = (1 + factor_daily_return[factor]).prod() - 1

        # 计算年化收益率：假设一年有244个交易日，将总收益率换算为年化
        annual_return = (1 + factor_daily_return[factor]).prod() ** (244 / len(factor_daily_return)) - 1

        # 计算年化波动率：日收益率的标准差乘以交易日数的平方根
        annual_volatility = factor_daily_return[factor].std() * np.sqrt(244)

        # 计算最大回撤
        # 首先计算累计收益率
        cumulative_return = (1 + factor_daily_return[factor]).cumprod()
        # 然后计算当前收益与历史最高收益的比值减1，取最小值
        max_drawdown = (cumulative_return / cumulative_return.cummax() - 1).min()

        # 设定无风险收益率为2%
        risk_free_rate = 0.02

        # 计算Sharpe比率：超额收益除以波动率
        sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

        # 计算Calmar比率：年化收益率除以最大回撤的绝对值
        calmar_ratio = annual_return / abs(max_drawdown)

        # 计算Sortino比率：只考虑下行风险的Sharpe比率变种
        # 先筛选出负收益的日期
        downside_returns = factor_daily_return[factor][factor_daily_return[factor] < 0]
        # 计算超额收益除以下行风险
        sortino_ratio = (annual_return - risk_free_rate) / (downside_returns.std() * np.sqrt(244))

        # 计算日胜率：正收益天数占比
        daily_win_rate = (factor_daily_return[factor] > 0).mean()

        # 计算日盈亏比：平均每日盈利与平均每日亏损的比值
        daily_profit_loss_ratio = factor_daily_return[factor][factor_daily_return[factor] > 0].mean() / abs(
            factor_daily_return[factor][factor_daily_return[factor] < 0].mean())

        # 将日数据重采样为周数据和月数据
        weekly_df = factor_daily_return[factor].resample('W').sum()
        monthly_df = factor_daily_return[factor].resample('M').sum()

        # 计算周胜率：正收益周数占比
        weekly_win_rate = (weekly_df > 0).mean()

        # 计算周盈亏比：平均每周盈利与平均每周亏损的比值
        weekly_profit_loss_ratio = weekly_df[weekly_df > 0].mean() / abs(weekly_df[weekly_df < 0].mean())

        # 计算月胜率：正收益月数占比
        monthly_win_rate = (monthly_df > 0).mean()

        # 计算月盈亏比：平均每月盈利与平均每月亏损的比值
        monthly_profit_loss_ratio = monthly_df[monthly_df > 0].mean() / abs(monthly_df[monthly_df < 0].mean())

        # 换手率（未实现）
        # turnover_rate =

        # 处理因子名称，移除"日收益率"字样
        factor_abbr = factor.replace("日收益率", "")
        
        # 将所有计算出的指标存入结果DataFrame
        indicators_df.loc[:, factor_abbr] = [
            "{:.2f}%".format(absolute_return * 100),  # 绝对收益率，百分比格式
            "{:.2f}%".format(annual_return * 100),    # 年化收益率，百分比格式
            "{:.2f}%".format(annual_volatility * 100), # 年化波动率，百分比格式
            "{:.2f}%".format(max_drawdown * 100),     # 最大回撤，百分比格式
            "{:.2f}".format(sharpe_ratio),            # Sharpe比率
            "{:.2f}".format(calmar_ratio),            # Calmar比率
            "{:.2f}".format(sortino_ratio),           # Sortino比率
            "{:.2f}%".format(daily_win_rate * 100),   # 日胜率，百分比格式
            "{:.2f}".format(daily_profit_loss_ratio), # 日盈亏比
            "{:.2f}%".format(weekly_win_rate * 100),  # 周胜率，百分比格式
            "{:.2f}".format(weekly_profit_loss_ratio), # 周盈亏比
            "{:.2f}%".format(monthly_win_rate * 100), # 月胜率，百分比格式
            "{:.2f}".format(monthly_profit_loss_ratio) # 月盈亏比
            # turnover_rate  # 换手率（未实现）
        ]
    # 返回包含所有因子评估指标的DataFrame
    return indicators_df


In [13]:
metrics = factor_evaluation(df_concat)

C:\Users\Yuuuu\AppData\Local\Temp\ipykernel_64660\1308368848.py:79: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_df = factor_daily_return[factor].resample('M').sum()
C:\Users\Yuuuu\AppData\Local\Temp\ipykernel_64660\1308368848.py:79: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_df = factor_daily_return[factor].resample('M').sum()
C:\Users\Yuuuu\AppData\Local\Temp\ipykernel_64660\1308368848.py:79: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_df = factor_daily_return[factor].resample('M').sum()
C:\Users\Yuuuu\AppData\Local\Temp\ipykernel_64660\1308368848.py:79: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_df = factor_daily_return[factor].resample('M').sum()
C:\Users\Yuuuu\AppData\Local\Temp\ipykernel_64660\1308368848.py:79: FutureWarnin

In [14]:
metrics.rename(columns={"portfolio_daily_return_IFIH": "IF_IH", 
                        "portfolio_daily_return_ICIF": "IC_IF", 
                        "portfolio_daily_return_IMIC": "IM_IC", 
                        "portfolio_daily_return_IMIH": "IM_IH", 
                        "portfolio_daily_return": "组合4个品种",}, inplace=True)
metrics 

,IF_IH,IC_IF,IM_IC,IM_IH,组合4个品种
绝对收益率,4.15%,19.95%,13.88%,33.98%,17.98%
年化收益率,3.61%,17.17%,11.99%,29.03%,15.49%
年化波动率,4.62%,12.15%,8.10%,18.20%,8.42%
最大回撤,-2.58%,-5.95%,-6.53%,-13.95%,-5.07%
Sharpe比率,0.35,1.25,1.23,1.48,1.60
Calmar比率,1.40,2.88,1.84,2.08,3.06
Sortino比率,0.61,1.96,1.62,1.75,2.25
日胜率,48.93%,47.50%,49.29%,48.57%,56.79%
日盈亏比,1.00,1.15,1.07,1.07,1.05
周胜率,58.06%,53.23%,48.39%,56.45%,58.06%


In [20]:
metrics.to_csv(r"D:\江西财经大学\31-实习\期货策略\daily_futures_arbitrary_strategy\metrics.csv", index=True)